In [2]:
pip install torch transformers bitsandbytes peft trl datasets accelerate scipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from trl import SFTTrainer
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

## Setup and Config

In [4]:
MODEL_ID = "state-spaces/mamba-1.4b-hf"
OUTPUT_DIR = "./babilong_mamba_finetune"

# Task Setup
TASK_NAME = "qa1"
SPLIT_LENGTH = "0k"  # Ideal für den Start auf der 3080

# QLoRA Config (Speicher sparen)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16, # RTX 3080 Feature
    bnb_4bit_use_double_quant=True,
)
# 2. Modell laden mit der Config
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config, # <--- DAS IST ENTSCHEIDEND
    device_map="auto",
    trust_remote_code=True # Mamba braucht das oft noch
)

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation for mamba-ssm and install the kernels library using `pip install kernels` or https://github.com/Dao-AILab/causal-conv1d for causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# LoRA Config für MAMBA
# Mamba hat spezifische Layer-Namen. 'all-linear' ist hier der sicherste Weg,
# um 'in_proj', 'out_proj', 'x_proj' und 'dt_proj' automatisch zu erwischen.
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["in_proj", "x_proj", "dt_proj"]
)

In [6]:
from transformers import AutoTokenizer

MODEL_ID = "state-spaces/mamba-1.4b-hf"

print(f"Lade Tokenizer für {MODEL_ID}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# Fix für Mamba / GPT-NeoX Tokenizer:
# Da kein 'pad_token' definiert ist, nutzen wir das End-of-Sentence Token.
tokenizer.pad_token = tokenizer.eos_token

# Für Training mit SFTTrainer ist "right" padding Standard
tokenizer.padding_side = "right" 

print(f"Tokenizer geladen. Pad Token ID: {tokenizer.pad_token_id}")

Lade Tokenizer für state-spaces/mamba-1.4b-hf...
Tokenizer geladen. Pad Token ID: 0


## Load Dataset

In [7]:
from datasets import load_dataset

# ==========================================
# DATASET LADEN & FORMATIEREN
# ==========================================

print(f"Lade BABILong {TASK_NAME} ({SPLIT_LENGTH})...")

# 1. Laden des spezifischen Splits (z.B. split='qa1')
# Das lädt NUR die Daten für diesen Task. Wir müssen nicht mehr filtern.
dataset = load_dataset("RMT-team/babilong", SPLIT_LENGTH, split=TASK_NAME)

# DEBUG: Zeige uns, welche Spalten wirklich da sind (vermeidet KeyErrors in der Zukunft)
print(f"Verfügbare Spalten: {dataset.column_names}")

# 2. Optional: Nur einen kleinen Teil zum Testen nutzen (Auskommentieren für echtes Training)
# dataset = dataset.select(range(100)) 

def formatting_prompts_func(example):
    """
    Formatierungsfunktion V3 (Final Fix).
    Unterscheidet sauber zwischen Batch (Liste) und Single (String),
    damit der SFTTrainer nicht über Datentypen stolpert.
    """
    # 1. Prüfen: Haben wir einen Batch (Liste von Inputs) oder ein einzelnes Item?
    # Wir prüfen 'input', da dies im Dataset vorhanden ist.
    is_batch = isinstance(example['input'], list)
    
    if is_batch:
        # === BATCH MODUS ===
        output_texts = []
        for i in range(len(example['input'])):
            text = (
                f"Context: {example['input'][i]}\n\n"
                f"Question: {example['question'][i]}\n\n"
                f"Answer: {example['target'][i]}"
            )
            output_texts.append(text)
        return output_texts # Rückgabe: Liste von Strings
        
    else:
        # === SINGLE SAMPLE MODUS ===
        # Hier dürfen wir KEINE Liste zurückgeben, sondern nur den nackten String!
        text = (
            f"Context: {example['input']}\n\n"
            f"Question: {example['question']}\n\n"
            f"Answer: {example['target']}"
        )
        return text # Rückgabe: Ein einzelner String

print("Dataset erfolgreich geladen und bereit.")

Lade BABILong qa1 (0k)...
Verfügbare Spalten: ['target', 'input', 'question']
Dataset erfolgreich geladen und bereit.


In [8]:
dataset[0]

{'target': 'bathroom',
 'input': 'John travelled to the hallway. Mary journeyed to the bathroom. Daniel went back to the bathroom. John moved to the bedroom.',
 'question': 'Where is Mary? '}

## Training Config and start

In [9]:
from trl import SFTTrainer, SFTConfig

OUTPUT_DIR = "./babilong_mamba_finetune"

# 1. Die Konfiguration (Hier kommen jetzt die Parameter rein!)
sft_config = SFTConfig(
    output_dir=OUTPUT_DIR,
    #max_seq_length=4096,             # <--- HIER muss es stehen
    packing=False,                   # <--- HIER muss es stehen
    dataset_text_field="text",       # Dummy-Feld (wird ignoriert wegen formatting_func, muss aber oft da sein)
    
    # Standard Training Arguments
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    num_train_epochs=1,
    warmup_ratio=0.03,
    fp16=False,
    bf16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    group_by_length=False,
    disable_tqdm=False,  # <--- Das erzwingt den Ladebalken (ist aber meist Default)
)

print("Initialisiere SFTTrainer...")
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    #tokenizer=tokenizer,
    args=sft_config,       # <--- Wir übergeben die Config hier
    # WICHTIG: Hier unten DARF KEIN max_seq_length oder packing mehr stehen!
)

print("🚀 Starte Training...")
trainer.train()

print(f"Training beendet. Speichere Adapter in {OUTPUT_DIR}...")
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Initialisiere SFTTrainer...


Applying formatting function to train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

...

🚀 Starte Training...


D:\Users\Louis\PycharmProjects\Master_thesis\Babilong_Benchmark\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,9.492000
20,4.585200


Training beendet. Speichere Adapter in ./babilong_mamba_finetune...


('./babilong_mamba_finetune\\tokenizer_config.json',
 './babilong_mamba_finetune\\special_tokens_map.json',
 './babilong_mamba_finetune\\tokenizer.json')